# 备选方法

In [1]:
import pandas as pd
import akshare as ak


## 文件间引用

In [ ]:
from IPython.display import IFrame
IFrame(src='model-analyzing.ipynb', width='100%', height='600px')

## 获取数据

In [3]:
# 获取制定商品期货的历史数据
futures_main_sina_hist = ak.futures_main_sina(symbol="RB0", start_date="20230101", end_date="20231027")
futures_zh_spot_df = ak.futures_zh_spot(symbol='RB2401', market="CF", adjust='0')
futures_zh_spot_df

symbol    time    open    high     low  current_price  bid_price  \
0  螺纹钢2401  230000  3928.0  3944.0  3911.0         3930.0     3929.0   

   ask_price  buy_vol  sell_vol       hold  volume  avg_price  last_close  \
0     3930.0       63       154  1566923.0  477386        0.0         0.0   

   last_settle_price  
0             3957.0

In [ ]:
# 期货基础信息
futures_comm_info_df = ak.futures_comm_info(symbol="所有")
futures_comm_info_df.to_excel('data/common_info.xlsx')

In [ ]:
# 获得期货品种
futures_symbol_mark_df = ak.futures_symbol_mark()

In [ ]:
import warnings
warnings.filterwarnings('ignore')
# 从akshare获取基差数据
# 基差数据截止到2023.9.14 ？

# stock_zh_a_minute_df = ak.stock_zh_a_minute(symbol='sz000876', period='1', adjust="qfq")
#df1 = ak.futures_spot_price_daily(start_day='20230101',end_day='20230914', vars_list=['RB'])
#df2 = ak.futures_spot_price_daily(start_day='20230916',end_day='20231020', vars_list=['RB'])
futures_spot_price_df = ak.futures_spot_price("20231106")

## Plotly用法

In [2]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

df_rb_basis = pd.read_csv('data/rb-main-basis-20231028.csv')
df_rb_basis['基差'] = df_rb_basis['现货'] - df_rb_basis['期货']

In [4]:
# 使用GraphicObject绘制多曲线
fig_main = make_subplots(specs=[[{"secondary_y": True}]])
# 不同颜色设置的方法
fig_future_price = go.Scatter(x=df_rb_basis['日期'], y=df_rb_basis['期货'], stackgroup='one', name='期货价格', marker_color='lightgray')
#fig_future_price = go.Scatter(x=df_rb_basis['日期'], y=df_rb_basis['期货'], stackgroup='one', name='期货价格', marker_color='rgb(15, 83, 109)')
#fig_future_price = go.Scatter(x=df_rb_basis['日期'], y=df_rb_basis['期货'], stackgroup='one', name='期货价格', marker_color='#af0110')

fig_basis_rate = go.Scatter(x=df_rb_basis['日期'], y=df_rb_basis['基差率'], name='基差率')
fig_main.add_trace(fig_future_price)
fig_main.add_trace(fig_basis_rate, secondary_y=True)

# 标注日期范围
fig_main.update_layout(
    shapes = [dict(  # 显示形状的位置和线宽等信息
        x0='2021-06-01', x1='2022-05-13',  # x的取值
        y0=0, y1=1,  # y的取值
        xref='x', yref='paper',
        line_width=1),
        dict(  # 显示形状的位置和线宽等信息
        x0='2022-08-01', x1='2022-09-13',  # x的取值
        y0=0, y1=1,  # y的取值
        xref='x', yref='paper',
        line_width=1)],
    annotations=[dict(   #  备注信息
        x='2022-06-01', y=0.05,
        xref='x', yref='paper',
        showarrow=False,
        xanchor='left',
        text='下降阶段')]
)
fig_main.layout.xaxis.title="日期"
fig_main.layout.yaxis.title="期货价格"
fig_main.layout.yaxis2.title="°基差率"
fig_main.layout.title="期货价格-基差分析"

# 去除休市的日期，保持连续
import datetime
trade_date = ak.tool_trade_date_hist_sina()['trade_date']
trade_date = [d.strftime("%Y-%m-%d") for d in trade_date]
dt_all = pd.date_range(start=df_rb_basis['日期'].iloc[0],end=df_rb_basis['日期'].iloc[-1])
dt_all = [d.strftime("%Y-%m-%d") for d in dt_all]
dt_breaks = list(set(dt_all) - set(trade_date))


# 数据筛选    
fig_main.update_xaxes(
    title_text = 'date',
    rangebreaks=[dict(values=dt_breaks)],
    rangeslider_visible = True, # 下方滑动条缩放
    rangeselector = dict(
        # 增加固定范围选择
        buttons = list([
            dict(count = 1, label = '1M', step = 'month', stepmode = 'backward'),
            dict(count = 6, label = '6M', step = 'month', stepmode = 'backward'),
            dict(count = 1, label = '1Y', step = 'year', stepmode = 'backward'),
            dict(count = 1, label = 'YTD', step = 'year', stepmode = 'todate'),
            dict(step = 'all')])))

fig_main.show()

In [6]:
# 使用express对象绘图（无法多图）
df_rb_basis = pd.read_csv('data/rb-main-basis-20231028.csv')
df_rb_basis['基差'] = df_rb_basis['现货'] - df_rb_basis['期货']
df_rb_basis['基差颜色'] = df_rb_basis['基差'] > 0

# fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
#                    vertical_spacing=0.01, subplot_titles=('期货价格', '基差'), 
#                    row_width=[0.3, 0.7])
fig_main = make_subplots(specs=[[{"secondary_y": True}]])
fig_future_price = px.area(df_rb_basis, x='日期', y='期货')
#fig_basis_rate = px.line(df_rb_basis, x='日期', y = '基差率',  color_discrete_sequence=["#F05023"])
fig_basis_rate = px.bar(df_rb_basis, x='日期', y = '基差率',  color='基差颜色')
fig_basis_rate.update_traces(yaxis="y2",showlegend=True,name='基差率')
fig_main.add_traces(fig_future_price.data + fig_basis_rate.data)

fig_main.layout.xaxis.title="日期"
fig_main.layout.yaxis.title="期货价格"
fig_main.layout.yaxis2.title="°基差率"
fig_main.layout.title="期货价格-基差分析"

fig_main.show()
# fig.add_trace(fig_main, row = 1, col = 1)
# fig_basis = go.Bar(x=df_rb_basis['日期'], y = df_rb_basis['基差'])
# fig.add_trace(fig_basis, row = 2, col = 1)
# fig.show()

In [7]:
# 按年度绘制多曲线
df_rb_basis['日期'] = pd.to_datetime(df_rb_basis['日期'], format="%Y-%m-%d")
df_rb_basis['年度'] = df_rb_basis['日期'].dt.year
df_rb_basis['年内日期'] = df_rb_basis['日期'].dt.strftime('1900-%m-%d')

basis_years = set(df_rb_basis['年度'])
f = go.Figure()
for y in basis_years:
    f.add_trace(go.Line(x=df_rb_basis[df_rb_basis['年度']==y]['年内日期'],
                        y=df_rb_basis[df_rb_basis['年度']==y]['基差率']))
f.show()


/Volumes/Repository/app/miniconda3/lib/python3.10/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [8]:
import plotly.offline as pyo

df_rb_basis['日期'] = pd.to_datetime(df_rb_basis['日期'])
df_grouped = df_rb_basis.groupby(df_rb_basis['日期'].dt.year)
# 创建一个空列表来存储每个年份的图表数据
data = []
# 遍历每个年份的数据
for year, group in df_grouped:
    # 创建一个散点图
    trace = go.Scatter(
        x=group['日期'].dt.strftime('1900-%m-%d'),
        y=group['基差率'],
        mode='lines',
        name=str(year)
    )
    # 将散点图添加到数据列表中
    data.append(trace)

# 创建布局
layout = go.Layout(
    title='基差率曲线',
    xaxis=dict(title='日期'),
    yaxis=dict(title='基差率'),
    hovermode='closest'
)

# 创建图表
fig = go.Figure(data=data, layout=layout)

# 显示图表
pyo.iplot(fig)

## Seaborn用法

In [9]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings


In [10]:
# 忽略警告
warnings.filterwarnings('ignore')
# 根据系统设置中文显示
MAC_OS = False
if MAC_OS:
    plt.rcParams['font.family'] = ['Arial Unicode MS']
else:
      rc = {'font.sans-serif': 'SimHei', 'axes.unicode_minus': False}
      sns.set(context='notebook', style='ticks', rc=rc)

## PygWalker用法

In [11]:
import pygwalker as pyg

In [ ]:
# 使用PygWalker进行可视化分析
pyg.walk(df_rb_basis, spec='./data/basis.jason')

In [5]:
import dash_mantine_components as dmc
from dash import Dash, html

app = Dash(__name__)

head_bar = dmc.Paper(
    children=["Future Nexus"],
    shadow="xs",
    radius="xs",
    p="xs",
    withBorder=True,
)

paper = dmc.Paper(
    children=["Paper is the most basic ui component. Use it to create cards, dropdowns, modals and other components that require background with shadow"],
    shadow="xs",
    p="xl",
    # withBorder=True,
)

header = dmc.Header(
    height=40, 
    children=[
        # dmc.Text("Company Logo"),
        head_bar,
    ], 
    style={"backgroundColor": "#9c86e2"}
)

footer = dmc.Footer(
    height=30, children=[dmc.Text("Company Logo")], style={"backgroundColor": "#9c86e2"}
)

style = {
    "border": f"1px solid {dmc.theme.DEFAULT_COLORS['indigo'][4]}",
    "textAlign": "center",
}

main_context = dmc.Grid(
    children=[
        dmc.Col(paper, span=4),
        dmc.Col(html.Div("2", style=style), span=4),
        dmc.Col(html.Div("3", style=style), span=4),
    ],
    gutter="5",
)

sidebar = dmc.Aside(
    p="md",
    width={"base": 150},
    # height=500,
    fixed=True,
    position={"left": 0, "top": 60},
    children=[
        dmc.Text("Link1"),
        dmc.Text("Link2"),
        dmc.Text("Link3"),
        dmc.Text("Link4"),
    ],
)

app.layout = html.Div(
    [
        dmc.Stack(
            children=[
                head_bar,
                dmc.Grid(
                    [
                        dmc.Col(sidebar, span=3),
                        dmc.Col(main_context, span=9),
                    ],
                    gutter=0,
                ),
                footer,
            ],
            spacing=2,
        )
    ]
)

if __name__ == "__main__":
    app.run_server()